In [2]:
from scipy.optimize import fsolve

# Returns as per Chapeter 7.1.2 example
def f(x):
    return 10+10/(1+x)-19.8/(1+x)**2
root = fsolve(f, 0) # 0 is the initial guess.
print('The Dollar-Weighted Return is ', root.item())

The Dollar-Weighted Return is  -0.006681547693192119
[-0.00668155]


In [3]:
# Calculating returns based on a list of cashflows.
from datetime import datetime

cashflows = [10, 10, -19.8]
dates = ['2020-01-01', '2021-01-01', '2022-01-01']
dates = [datetime.strptime(dt, "%Y-%m-%d") for dt in dates]

def f(x, *args):
    cashflows, dates = args
    sumPVs = 0
    for i in range(len(cashflows)):
        Y = (dates[i] - dates[0]).days/365
        sumPVs += cashflows[i]/(1+x)**Y
    return sumPVs
root = fsolve(f, 0, args=(cashflows, dates))
print('The Dollar-Weighted Return is ', root.item())

The Dollar-Weighted Return is  -0.0066754853766419105


In [6]:
# Return calcualtion - SUSA ESG example

import pandas_datareader as pdr
import yfinance as yf
import statsmodels.api as sm
import math
 
start='2005-02-01'
end = '2022-09-30'

rf_daily = pdr.DataReader('DGS1MO', 'fred', start=start, end=end)
ESG_prices = yf.download('SUSA', start=start, end=end)['Adj Close']
mkt_prices = yf.download('SPY', start=start, end=end)['Adj Close']

rf_monthly = rf_daily.resample('MS').first()

ESG_prices_monthly = ESG_prices.resample('MS').first()
mkt_prices_monthly = mkt_prices.resample('MS').first()

ESG_returns_monthly = ESG_prices_monthly.pct_change()
mkt_returns_monthly = mkt_prices_monthly.pct_change()

y = (ESG_returns_monthly - rf_monthly['DGS1MO']/100/12).dropna()
X = (mkt_returns_monthly - rf_monthly['DGS1MO']/100/12).dropna()

# Add a column of one’s to the right-hand side of the regression using the command sm.add_constant().
# The regression coefficient on the column of ones represents the constant or intercept of the regression,
# which is the alpha we are trying to compute.
X = sm.add_constant(X)

model = sm.OLS(y,X)
results = model.fit()

print(results.summary())

# Information ratio of the fund, with the S&P500 as the benchmark
IR = (y-X[0]).mean()/(y-X[0]).std()*math.sqrt(12)

print('The Information Ratio is ', IR)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.981
Model:                            OLS   Adj. R-squared:                  0.981
Method:                 Least Squares   F-statistic:                 1.088e+04
Date:                Mon, 15 Jul 2024   Prob (F-statistic):          3.28e-182
Time:                        17:59:22   Log-Likelihood:                 751.46
No. Observations:                 211   AIC:                            -1499.
Df Residuals:                     209   BIC:                            -1492.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0004      0.000     -0.762      0.4